In [ ]:
###########################################################
#                                                         #
#               TRAITEMENT SIMUS FORTRAN                  #
#                     STATISTIQUES                        #
#                                                         #
#                                                         #
#                          04/2021     Adrien Poindron    #
###########################################################

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
matplotlib.rcParams['figure.figsize'] = [10.5,7]
matplotlib.rcParams.update({'font.size': 25})

# Imports

In [71]:
# %pylab

import numpy
import matplotlib
from matplotlib.patches import Ellipse
from matplotlib import pylab, mlab, pyplot
from mpl_toolkits.mplot3d import Axes3D
np = numpy
plt = pyplot

matplotlib.rcParams.update({'font.size': 21})
# matplotlib.rcParams["figure.dpi"] = 300

from IPython.display import display
from IPython.core.pylabtools import figsize, getfigs

from pylab import *
from numpy import *

from scipy.optimize import curve_fit

In [72]:
import os
from os import listdir
from os.path import isfile, join

import tkinter as tk
from tkinter import filedialog
%matplotlib qt

import matplotlib.patches as patches

from pathlib import Path
from matplotlib import gridspec
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

In [73]:
from natsort import natsorted   # natural sort marche de ouf
import re                       # find '/' in strings
import timeit
cm = pylab.get_cmap('summer')   # colormap par défaut color=cm(k/len(array))

In [93]:
# Fonctions persos de Jofre et Adrien

# %run function_jofre.ipy
%run "../Functions/210426_stopping_power.py"

In [75]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide

r0 = 2.5e-3 # 2.5e-3   # rayon piège Ca+
d0 = 4e-3/2            # longueur piège Ca+

Omega = 2.05e6*2*pi # 2.047e6

bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23          # écrantage piège réel GiantMol
wzLC = (2*pi*90806.9982303)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.270471


# Sélection fichier

In [95]:
## GUI for data loading
# Select one data file all the way down to the directories
# Temp_SimuType0_N01024_Vrf0064_Udc0.5000D+00V_D1.0_S1.0RFG.dat

file_cfg, slash_cfg, all_subdir = load_gui(filter_nocomplete=1)

file_path = file_cfg[0]
dir_path = file_cfg[1]
work_rep = file_cfg[2]
filename = file_cfg[3]

myslashpos = slash_cfg[0]
slashcond = slash_cfg[1]

> Répertoire : /home/adrian/RemoteFS/Rivendel/20210426/EGMol07/Try00
> Filename : Temp_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG50eV.dat
> myslashpos | [0, 5, 12, 21, 30, 39, 47]
> slashcond | 5
Points deleted because they were not complete []   0 pt(s)
Total number of data directories 70


In [96]:
## Condition for data
points_and_coord, condition_parameters = simu_conditions(all_subdir, myslashpos, slashcond, filename)

['EGMol07', 'EGMol08', 'EGMol09', 'EGMol10', 'EGMol11', 'EGMol12', 'EGMol13']
> condition names ['EGMol']
> number of points 7
> N_ions = 1024
> e_GMol = 50


In [97]:
## Data loading (1D arrays)
data_name, num_runs, PMandT, Gmol_data, r_LC_clip, dim_nu \
= data_retrieve(all_subdir, points_and_coord,condition_parameters,slash_cfg,mode='stopping_power') 

data0 = data_name[0]
data2 = data_name[1]
data4 = data_name[2]

PMvar = PMandT[0]
Tvar = PMandT[1]

deltaEc = Gmol_data[0]
deltaEcRel = Gmol_data[1]
t_c = Gmol_data[2]

> Points | 7
> Simulations pour chaque point | ['Try00', 'Try01', 'Try02', 'Try03', 'Try04', 'Try05', 'Try06', 'Try07', 'Try08', 'Try09']
Hello
Point n° 0
00 - 00  >  /home/adrian/RemoteFS/Rivendel/20210426/EGMol07/Try00/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG
['Temp_SimuType2_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG50eV.dat', 'Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG.dat', 'Temp_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG50eV.dat']
00 - 01  >  /home/adrian/RemoteFS/Rivendel/20210426/EGMol07/Try01/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG
['Temp_SimuType2_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG50eV.dat', 'Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG.dat', 'Temp_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG50eV.dat']
00 - 02  >  /home/adrian/RemoteFS/Rivendel/20210426/EGMol07/Try02/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG
['Temp_SimuType2_01_N01024_Vrf0064_Udc0.2000D+01V_D1.

02 - 05  >  /home/adrian/RemoteFS/Rivendel/20210426/EGMol09/Try05/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG
['Temp_SimuType2_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG8eV.dat', 'Temp_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG8eV.dat', 'Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG.dat']
02 - 06  >  /home/adrian/RemoteFS/Rivendel/20210426/EGMol09/Try06/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG
['Temp_SimuType2_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG8eV.dat', 'Temp_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG8eV.dat', 'Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG.dat']
02 - 07  >  /home/adrian/RemoteFS/Rivendel/20210426/EGMol09/Try07/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG
['Temp_SimuType2_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG8eV.dat', 'Temp_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG8eV.dat', 'Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG.dat']
02 -

Point n° 5
05 - 00  >  /home/adrian/RemoteFS/Rivendel/20210426/EGMol12/Try00/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG
['Temp_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG2eV.dat', 'Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG.dat', 'Temp_SimuType2_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG2eV.dat']
05 - 01  >  /home/adrian/RemoteFS/Rivendel/20210426/EGMol12/Try01/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG
['Temp_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG2eV.dat', 'Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG.dat', 'Temp_SimuType2_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG2eV.dat']
05 - 02  >  /home/adrian/RemoteFS/Rivendel/20210426/EGMol12/Try02/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG
['Temp_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG2eV.dat', 'Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG.dat', 'Temp_SimuType2_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG2eV

In [98]:
# Where SNR for each try in each  are compared to a threshold
# If SNR > some_threshold , we count it as a detection
# Then for each point the ratio number_of_detection/number_of_try is calculated
# This detection efficiency 

SNR_trig_threshold = 1
shapevar = ((len(points_and_coord), len(num_runs)))
trig_detection = np.zeros((shapevar))
for i in range(len(points_and_coord)):
    SNR = Gmol_data[2][i]
    for k,m in enumerate(SNR):
        if m > SNR_trig_threshold:
            trig_detection[i,k] = 1

print(mean(trig_detection,axis=1))

print(len(trig_detection))

# Where is written Detuning and how much it is
temp_pos = filename.find('0RFG')
Detuning = filename[temp_pos-7:temp_pos-4]
print(Detuning)

[0. 0. 0. 0. 0. 0. 0.]
7
1.0


In [99]:
# Voltages associated with each condition
# In names and files each voltage is associated with numbers
# For instance on epoint is labeled DC03_RF18
# You need to know which voltage is DC03 and RF18

# You have to look on the Fortran code, example :
#    !dec$ elseif defined(Vrf18)
#    double precision  , parameter :: V_rf  = 7.179d1
#    character(len=100), parameter :: str_extra3 = 'q=0.666'
# Here RF18 is 71.79 V, q = 0.666

Udc = [0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 6, 7, 7.5, 3.33, 3.66, 4.5, 5.33, 5.66, 6, 6.5]
Urf = [10.77, 16.15, 21.54, 26.92, 32.31, 37.69, 43.08, 48.46, 53.85, 59.23, 64.61,
       70.00, 75.38, 80.77, 61.02, 62.82, 66.41, 68.20, 71.79, 55.64, 57.43]
len(Udc),len(Urf)

(20, 21)

In [82]:
# recovery temperature
# calculate T final SimuType0
# Determination error on temperature
# with measurement of the variation in the simulations

my_order = [0,2,1]

# retrieve time array
point_numb = 3
point_try = 0
plot_subdir = all_subdir[point_numb*13+point_try%13] # point address
onlyfiles = [f for f in listdir(plot_subdir) if isfile(join(plot_subdir, f)) and not "xva" in f and ".dat" in f]

total_len = 0
cut_pts = [0]
simu_time = []
for j,k in enumerate(my_order):
    ta, te, ti, to = load_T_and_PM_simu(plot_subdir+'/'+'Temp'+sort(onlyfiles)[k].strip('.dat')[4:])
    total_len += len(ta)
    # print(len(ta))
    cut_pts.append(total_len)
    simu_time.extend(ta)
print(temp_pts)

# retrieve temperatures
T_aux_all = np.zeros((len(points_and_coord),13,total_len,3))
for point_numb,point_name in enumerate(points_and_coord): # DCxx_RFyy [xx,yy]
    for point_try in range(13):
        plot_subdir = all_subdir[point_numb*13+point_try%13] # point address
        onlyfiles = [f for f in listdir(plot_subdir) if isfile(join(plot_subdir, f)) and not "xva" in f and ".dat" in f]
        for j,k in enumerate(my_order):
            ta, te, ti, to = load_T_and_PM_simu(plot_subdir+'/'+'Temp'+sort(onlyfiles)[k].strip('.dat')[4:])
            T_aux_all[point_numb,point_try,cut_pts[j]:cut_pts[j+1],:] = ti
    print(point_numb)
        # tt, T_CM, T_aux, PM

NameError: name 'temp_pts' is not defined

# Plots
Nuage d'ions (XYZ) et température

## Visualise one specific point, and one case

# One try study

In [83]:
# plot ion cloud XYZ

print(plot_subdir)
# pick the right xyz files
onlyfiles = [f for f in listdir(plot_subdir) if isfile(join(plot_subdir, f)) and not "xva" in f and ".dat" in f]
file_name = '{}/xva{}'.format(plot_subdir,sort(onlyfiles)[0].strip('.dat')[4:])
plot_XYZ(file_name,fig_name='XYZ_point'+str(plot_point)+'_try'+str(plot_try),fig_title='XYZ')
auxr_LC,auxv_LC,auxa_LC = load_xyz_init_bin_DP(file_name)
len(auxr_LC[0])

/home/adrian/RemoteFS/Rivendel/20210426/EGMol10/Try09


ValueError: too many values to unpack (expected 3)

In [84]:
N_ions = 1024
n_dt = 500

In [85]:
# rep_name = 'K:/Universite_Simus/20210426/EGMol08/Try00/'
rep_name = '/home/adrian/RemoteFS/Rivendel/20210426/EGMol08/Try00/'
cond_name = 'Vrf0064_Udc0.2000D+01V'
# cond_name = 'Vrf0010_Udc0.5000D+02V'
print(rep_name)

# file = rep_name + 'Temp_SimuType0_N00128_Vrf0010_Udc0.5000D+00V_nt100'
file_SimuType0 = rep_name + f'Temp_SimuTypeQ_N{N_ions:05d}_' + cond_name + '_nt100'
print(file_SimuType0)
# time, T_CM, T_aux, samp_dE_dt, S_heating = load_T_and_dE_S_simu(file_SimuType0)
# print(samp_dE_dt)
# file_energies = rep_name + 'Energies_ndt100_vdt2'
# j_save_E, size_Energy_array, dE_dt = load_energies(file_energies,flag_print=1)
# full_dE_dt = reshape(dE_dt,(int(size_Energy_array),6),order='F')[:,3]
# print(full_dE_dt)

/home/adrian/RemoteFS/Rivendel/20210426/EGMol08/Try00/
/home/adrian/RemoteFS/Rivendel/20210426/EGMol08/Try00/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_nt100


In [86]:
str_load_Lan = rep_name + f'xva_3D_Harmo_N{N_ions:04d}_T500uK_F0.10D-19Kg_s_5'
# print(str_load_Lan)
# xva_pos = str_load_Lan.find('xva')
# print(str_load_Lan[:xva_pos]+'Langevin_cooling'+'.info')
# print(str_load_Lan+'.bin')
# plot_XYZ_Lan(str_load_Lan)

/home/adrian/RemoteFS/Rivendel/20210426/EGMol08/Try00/xva_3D_Harmo_N1024_T500uK_F0.10D-19Kg_s_5
/home/adrian/RemoteFS/Rivendel/20210426/EGMol08/Try00/Langevin_cooling.info
/home/adrian/RemoteFS/Rivendel/20210426/EGMol08/Try00/xva_3D_Harmo_N1024_T500uK_F0.10D-19Kg_s_5.bin
1024
b'xva_3D_Harmo_N1024_T500uK_F0.10D-19Kg_s_5'


/home/adrian/Documents/Programmes/Python/GiantMol/Functions/210426_stopping_power.py:165: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  aux_info = genfromtxt(str_load[:xva_pos]+'Langevin_cooling'+'.info',comments='%',dtype=None)


In [87]:
str_load_Ver = rep_name + f'xva_SimuTypeQ_N{N_ions:05d}_' + cond_name + '_D1.0_S1.0RFG'
print(str_load_Ver)
xva_pos = str_load_Ver.find('xva')
print(str_load_Ver[:xva_pos]+'Langevin_cooling'+'.info')
print(str_load_Ver+'.bin')
plot_XYZ(str_load_Ver)

/home/adrian/RemoteFS/Rivendel/20210426/EGMol08/Try00/xva_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG
/home/adrian/RemoteFS/Rivendel/20210426/EGMol08/Try00/Langevin_cooling.info
/home/adrian/RemoteFS/Rivendel/20210426/EGMol08/Try00/xva_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG.bin


In [100]:
## Data loading with hardcode names
eta = 1.5e-19

plot_point = 0
plot_try = 0
# str_eta_chiffre    = float(f'{eta[plot_point]:1.2e}'[:4])/10
# str_eta_puissance  = int(re.sub('e','D',f'{eta[plot_point]:1.2e}'[6:]))-1
# str_eta = f'{str_eta_chiffre:1.2f}'+'D-'+str(str_eta_puissance)
str_eta = '0.15D-18'
dirname_hc = str(data0[plot_point][plot_try])+'/'
datainit_hc = 'Temp_3D_Harmo_N1024_T500uK_F'+str(str_eta)+'Kg_s_5'
data0_hc    = '/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG'
data4_hc    = '/Temp_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG'
data2_hc    = '/Temp_SimuType2_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG'

print(dirname_hc)
print(datainit_hc)

/home/adrian/RemoteFS/Rivendel/20210426/EGMol07/Try00/
Temp_3D_Harmo_N1024_T500uK_F0.15D-18Kg_s_5


In [101]:
# Get T and fluo PM  All SimuType
onlyfiles = [f for f in listdir(rep_name) if isfile(join(rep_name, f)) and "SimuType" in f and ".dat" in f]
print(sort(onlyfiles))
only_init = [f for f in onlyfiles if "SimuTypeQ" in f]

['Temp_SimuType2_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG10eV.dat'
 'Temp_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG10eV.dat'
 'Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG.dat'
 'xva_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG10eV.dat']


In [132]:
# Fonctions persos de Jofre et Adrien

# %run function_jofre.ipy
%run "../Functions/210426_stopping_power.py"

In [133]:
onlyfiles[2].strip('.dat')

'Temp_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG10eV'

In [138]:
# Temperature and dE/dt

temps, temperature, fluo = plot_T_and_PM_Init_Inje_Evol(rep_name,onlyfiles[2].strip('.dat'),flag_plot=1,
                                    fig_name='popopo',
                                    xlim1=(-2,12),ylim1=(1e-5,8e4)) # ,ylim2=(-5e-12,5e-12)
# print(min(dE))
# savefig(rep_name+'T_dE')
# tt[1980],tt1[0]

len save_T [55440]
[5.05000000e-07 5.14176543e-05 7.38793448e-04 2.10273990e-05
 4.61211460e+01 1.50677356e+00 7.06664375e+01]
